In [1]:
from ipywidgets import SelectMultiple, IntSlider, Button, interact, fixed, FloatSlider
from model_scoring_func import get_all_names
from IPython.display import display, clear_output
from optimisation import min_max_df, my_g, property_results, formulation_result, opt_with_starts
from numpy import append
from pandas import DataFrame
from tinydb import TinyDB, Query

In [2]:
names = get_all_names()

In [3]:
sel_resp_var = SelectMultiple(options=list(names))

Select which response variables to constrain (use cntrl + click)

In [4]:
display(sel_resp_var)

In [5]:
con_list = list(sel_resp_var.value)
mins = min_max_df(con_list)['min'].values
maxs = min_max_df(con_list)['max'].values

lim_val = []

for name, mymin, mymax in zip(con_list, mins, maxs):
    lim_val.append(IntSlider(value=(mymax-mymin)/2, 
                             min=mymin, 
                             max=mymax, 
                             description=name))

ingredients = ['PVC', 'filler', 'FR', 'stabiliser', 'DINP', 'LDH', 'spherical_filler']
ingr_cost_in = [15.0, 8.0, 48.0, 45.0, 30.0, 40.0, 3.0]

ingr_cost_w = []
for name, cost in zip(ingredients, ingr_cost_in):
    ingr_cost_w.append(IntSlider(value=cost, min=0, max=60, description=name))

widget_list = append(lim_val, ingr_cost_w)

for w in widget_list:
    display(w)

In [6]:
a = 2

In [8]:
run_opt_button = Button(description='Run Optimisation')
display(run_opt_button)

def optimisation(b):
    q = Query()
    db = TinyDB('test.json')
    
    clear_output()
    print('Calculating...')
    
    con_limits = [w.value for w in lim_val]
    ingr_cost = [w.value for w in ingr_cost_w]
    
    cons = ({'type': 'eq',
             'fun': lambda x: sum(x) - 1},
            {'type': 'ineq', 
             'fun': my_g, 
             'args': [con_list, con_limits]})
    
    bnds = [[0.0, 1.0] for i in range(7)]
    bnds[0] = [0.4, 0.8]
    
    best_res = opt_with_starts(ingr_cost, bnds, cons)
    
    clear_output()
    print(best_res.message)
    print('Optimum Cost of Formulation: {} R/kg'.format(round(best_res.fun, 2)))
    
    form_df = formulation_result(best_res.x)
    display(form_df)
    
    prop_df = property_results(best_res.x, con_list, con_limits)
    display(prop_df)

    if not db.all():
        db.insert({'opt_x': list(best_res.x)})
        return
    
    db.update({'opt_x': list(best_res.x)}, q.opt_x.exists())
    
run_opt_button.on_click(optimisation)

Optimization terminated successfully.
Optimum Cost of Formulation: 17.37 R/kg


,mass_frac_%
PVC,62.28
filler,5.49
FR,6.07
stabiliser,1.25
DINP,12.46
LDH,0.00
spherical_filler,12.46


Calculating...


In [8]:
def model_comp_to_opt(filler, FR, stabiliser, DINP, LDH, sphFiller):
    phr = [100, filler, FR, stabiliser, DINP, LDH, sphFiller]
    ingredients = ['PVC', 'filler', 'FR', 'stabiliser', 'DINP', 'LDH', 'spherical_filler']

    mass_frac = [i/sum(phr) for i in phr]
    mass_frac_per = [round(100*i, 2) for i in mass_frac]
    
    check_db = TinyDB('test.json')

    mass_frac_opt = check_db.all()[0]['opt_x']
    phr_opt = [round(100*i/mass_frac_opt[0], 1) for i in mass_frac_opt]
    mass_frac_opt_per = [round(100*i, 2) for i in mass_frac_opt]

    df_form = DataFrame([phr, mass_frac_per, phr_opt, mass_frac_opt_per],
                        index=['phr', 'Mass Frac. %', 'Opt. phr', 'Opt. Mass Frac. %'], 
                        columns=ingredients).T

    df_pr = property_results(mass_frac, [], [], use_style=False)

    df_pr_opt = property_results(mass_frac_opt, [], [], use_style=False)

    df_pr = df_pr[['max', 'min', 'value_unscaled']]

    show_these = con_list

    df_pr_opt = df_pr_opt.loc[show_these]
    df_pr = df_pr.loc[show_these]

    df_pr['Optimum Values']= df_pr_opt['value_unscaled']
    df_pr = df_pr.round(2)

    display(df_pr, df_form)

In [10]:
opt_form_db = TinyDB('test.json')
mass_frac_opt = opt_form_db.all()[0]['opt_x']
phr_opt = [round(100*i/mass_frac_opt[0], 1) for i in mass_frac_opt]

w = interact(model_comp_to_opt,
             filler=FloatSlider(min=0, max=70, step=0.1, value=phr_opt[1]),
             FR=FloatSlider(min=0, max=20, step=0.1, value=phr_opt[2]),
             stabiliser=FloatSlider(min=2, max=10, step=0.1, value=phr_opt[3]),
             DINP=FloatSlider(min=20, max=70, step=0.1, value=phr_opt[4]),
             LDH=FloatSlider(min=0, max=15, step=0.1, value=phr_opt[5]),
             sphFiller=FloatSlider(min=0, max=20, step=0.1, value=phr_opt[6])
            )

,max,min,value_unscaled,Optimum Values
name,,,,
ConeCal peak_HRR_kWpm2,242.76,60.00,120.07,120.0
rheomix final_deg_time_min,346.90,12.60,69.87,70.0
tensile epsilon_break_%,114.77,9.82,50.04,50.0
tensile sigma_break_MPa,29.51,10.61,19.59,19.6


,phr,Mass Frac. %,Opt. phr,Opt. Mass Frac. %
PVC,100.0,53.62,100.0,53.64
filler,4.9,2.63,4.9,2.60
FR,20.0,10.72,20.0,10.73
stabiliser,2.0,1.07,2.0,1.07
DINP,35.4,18.98,35.4,18.97
LDH,4.2,2.25,4.2,2.26
spherical_filler,20.0,10.72,20.0,10.73
